# Import libs

In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
%matplotlib inline 
import sys  
reload(sys)  
sys.setdefaultencoding('utf8')
sys.getdefaultencoding()
import re
import os

# Get one file to check it out

In [12]:
soup = BeautifulSoup(open("/Users/plotti/swissinfo-en/Editorial/Content/Rubric/Business/2014/03/03/Fewer%20foreign%20bosses%20sign%20up%20to%20Swiss%20firms-163574985-38080398-detail.xml"), "lxml")

In [13]:
" ".join([s.get_text() for s in soup.select("p")])

u'Foreign power: American Joe Hogan, left, was the CEO of Zurich-based ABB from 2008-2013'

In [14]:
cats = [re.sub(r"[^a-z]", "",cat["href"].split("/")[-1].split(".")[0]) for cat in soup.select("subjecttaxonomy link")]
cats

[]

# Extract Info

In [8]:
texts = []
categories = []
for root, dirs, files in os.walk('/Users/plotti/swissinfo-en/Editorial/Content/Rubric/'):
     for file in files:
        with open(os.path.join(root, file), "r") as auto:
            #print("Working on %s" % auto)
            soup = BeautifulSoup(auto, "lxml")
            detail_text = " ".join([s.get_text() for s in soup.select("p")])
            if ((detail_text != "") & (len(detail_text) > 1000)):
                #print(len(detail_text))
                texts.append(detail_text)
                cats = [re.sub(r"[^a-zA-Z]", "",re.sub(".xml","",cat["href"].split("/")[-1].split(".")[0])) for cat in soup.select("subjecttaxonomy link")]
                categories.append(cats)

# Explode and turn into df

In [9]:
len(texts)

42139

In [10]:
out = []
for i,text in enumerate(texts):
    for category in categories[i]:
        out.append([category,text])

In [11]:
df = pd.DataFrame(out)
df.columns = ["tag","text"]
df["tag"] = pd.Categorical(df["tag"])
df['code'] = df["tag"].cat.codes
df.head(20)

,tag,text,code
0,demographics,PLACEHOLDER While serving in diplomatic posts ...,270
1,internationalrelations,PLACEHOLDER While serving in diplomatic posts ...,462
2,socialsciences,TOD 060313 PLACEHOLDER From the Arab Spring to...,752
3,internationalrelations,TOD 060313 PLACEHOLDER From the Arab Spring to...,462
4,nationalsecurity,TOD 060313 PLACEHOLDER From the Arab Spring to...,566
5,reliefandaidorganisation,TOD 060313 PLACEHOLDER From the Arab Spring to...,689
6,crimeClawandjustice,TOD 060313 PLACEHOLDER From the Arab Spring to...,248
7,ZPORCatholicChurch,"PLACEHOLDER As head of the Vatican state, the ...",93
8,internationalrelations,"PLACEHOLDER As head of the Vatican state, the ...",462
9,religionandbelief,"PLACEHOLDER As head of the Vatican state, the ...",690


In [12]:
len(df)

51755

# Save to CSV

In [15]:
df.to_csv("texts.csv")

In [16]:
dict( enumerate(df['tag'].cat.categories) )

{0: 'AIDS',
 1: 'Americanfootball',
 2: 'ArabspringMiddleEastandNorthAfricainturmoil',
 3: 'Argentdespotentats',
 4: 'AttacksinParis',
 5: 'Australianrulesfootball',
 6: 'Banksinturmoilfinancialsector',
 7: 'CEEEOEinterview',
 8: 'CabinetElection',
 9: 'Cannabis',
 10: 'Climatechange',
 11: 'Commodities',
 12: 'Directdemocracy',
 13: 'Dossiers',
 14: 'Dossiersen',
 15: 'EditorialSubjects',
 16: 'EducatingSwitzerland',
 17: 'Elections',
 18: 'Europe',
 19: 'EuropeanimmigrantsinSwitzerland',
 20: 'FactChecks',
 21: 'Faunaflora',
 22: 'FilmFestivalLocarno',
 23: 'FormulaOne',
 24: 'Frontpage',
 25: 'GoogleNewsStandout',
 26: 'Gotthardtunnel',
 27: 'IOC',
 28: 'ITCcomputersciences',
 29: 'Innovationandcleantech',
 30: 'InternationalGeneva',
 31: 'IslaminSwitzerland',
 32: 'LGBTIQinSwitzerland',
 33: 'LaprCAsidentiellefranCAaisevuedeSuisse',
 34: 'Longform',
 35: 'MakingSwitzerlandhome',
 36: 'MillenniumGoals',
 37: 'News',
 38: 'Opinion',
 39: 'PeoplePower',
 40: 'PortraitseriesFacesofSwit

## Waaayy to many categories, lets keep the 5 most used ones

In [1]:
df['tag'].value_counts().plot(kind="hist")

NameError: name 'df' is not defined

In [20]:
most_prominent_categories = list(df['tag'].value_counts()[0:4].keys())
most_prominent_categories

['internationalrelations',
 'economyCbusinessandfinance',
 'interiorpolicy',
 'artsCcultureandentertainment']

In [21]:
df_filtered = df[df['tag'].isin(most_prominent_categories)]
len(df_filtered)

11171

In [45]:
df_filtered.head(50)

,tag,text,code
1,internationalrelations,PLACEHOLDER While serving in diplomatic posts ...,462
3,internationalrelations,TOD 060313 PLACEHOLDER From the Arab Spring to...,462
8,internationalrelations,"PLACEHOLDER As head of the Vatican state, the ...",462
10,interiorpolicy,PLACEHOLDER Switzerland will contribute CHF10 ...,454
11,internationalrelations,PLACEHOLDER Switzerland will contribute CHF10 ...,462
12,internationalrelations,PLACEHOLDER At the conclusion of a ten-day Uni...,462
13,economyCbusinessandfinance,PLACEHOLDER At the conclusion of a ten-day Uni...,292
19,internationalrelations,also for dossier PLACEHOLDER The Swiss abroad ...,462
24,internationalrelations,PLACEHOLDER The Swiss foreign ministry has sai...,462
28,internationalrelations,PLACEHOLDER A Turkish court has handed down a ...,462


In [44]:
df_filtered['tag'].value_counts()

internationalrelations          3905
economyCbusinessandfinance      3340
interiorpolicy                  2198
artsCcultureandentertainment    1728
defenceequipment                   0
defendant                          0
earnings                           0
drugtrafficking                    0
defence                            0
drought                            0
defencecontract                    0
divorce                            0
distillerandbrewer                 0
disease                            0
discrimination                     0
disastergeneral                    0
disasterandaccident                0
disabled                           0
economicindicator                  0
diplomacy                          0
dietarysupplements                 0
diet                               0
dieselfuel                         0
designandengineering               0
demonstration                      0
demographics                       0
democracy                          0
e

# Prepare data for training

In [23]:
import json
from pathlib import Path
import re
import random

training_data = Path("fasttext_dataset_training.txt")
test_data = Path("fasttext_dataset_test.txt")

# What percent of data to save separately as test data
percent_test_data = 0.10


def strip_formatting(string):
    string = string.lower()
    string = re.sub(r"([.!?,'/()])", r" \1 ", string)
    return string

with training_data.open("w") as train_output, \
     test_data.open("w") as test_output:

    for i,item in df_filtered.iterrows():
        #print(item)
        if (item["tag"] != "") & (item["text"] != ""):
            tag = item["code"]
            text = item["text"].replace("\n", " ")
            text = strip_formatting(text)

            fasttext_line = unicode("__label__{} {}".format(tag, text),'utf-8')

            if random.random() <= percent_test_data:
                test_output.write(fasttext_line + "\n")
            else:
                train_output.write(fasttext_line + "\n")

# Train

In [24]:
!fasttext supervised -input fasttext_dataset_training.txt -output category_model 

# Test

In [25]:
!fasttext test category_model.bin fasttext_dataset_test.txt

In [26]:
!fasttext test category_model.bin fasttext_dataset_test.txt 3

# Results

- We've got 60% hit rate without tuning. 
- Thats enough since a text is mostly in multiple categories.
- Predicting 3 categories we get 90% recall rate. 

# Predict Something

- internationalrelations          3905
- economyCbusinessandfinance      3340
- interiorpolicy                  2198
- artsCcultureandentertainment    1728

In [54]:
r = dict( enumerate(df_filtered['tag'].cat.categories) )

In [55]:
out = commands.getstatusoutput("echo 'die schweizer wirtschaft floriert' | fasttext predict category_model.bin -")[1]
r[int(out.split("_")[-1])]

'economyCbusinessandfinance'

In [56]:
out = commands.getstatusoutput("echo 'die kunstwelt ist entzückt. die bilder verkaufen sich prima.' | fasttext predict category_model.bin -")[1]
r[int(out.split("_")[-1])]

'artsCcultureandentertainment'

In [60]:
out = commands.getstatusoutput("echo 'die deutsche bundeskanzlerin angela merkel einigte sich mit premierminister emanuelle macron aus frankreich auf einen EU-gipfel.' | fasttext predict category_model.bin -")[1]
r[int(out.split("_")[-1])]

'economyCbusinessandfinance'

In [61]:
out = commands.getstatusoutput("echo 'die berner abgeordneten verfügten über den haushalt 2019.' | fasttext predict category_model.bin -")[1]
r[int(out.split("_")[-1])]

'economyCbusinessandfinance'

# Train with tuning 

https://www.analyticsvidhya.com/blog/2017/07/word-representations-text-classification-using-fasttext-nlp-facebook/

In [65]:
!fasttext supervised -input fasttext_dataset_training.txt -output category_model -epoch 100 -lr 1.0

In [66]:
!fasttext test category_model.bin fasttext_dataset_test.txt

- 0.21 worse accuracy :(

In [27]:
!fasttext supervised -input fasttext_dataset_training.txt -output category_model -epoch 10 -lr 1.0 -wordNgrams 3

In [28]:
!fasttext test category_model.bin fasttext_dataset_test.txt

- 0.32 better accuracy

In [29]:
!fasttext supervised -input fasttext_dataset_training.txt -output category_model -epoch 10 -lr 1.0 -wordNgrams 2

In [30]:
!fasttext test category_model.bin fasttext_dataset_test.txt

- 0.35 

In [71]:
!fasttext supervised -input fasttext_dataset_training.txt -output category_model -epoch 30 -lr 1.0 -wordNgrams 2

In [72]:
!fasttext test category_model.bin fasttext_dataset_test.txt

- 0.238

In [73]:
!fasttext supervised -input fasttext_dataset_training.txt -output category_model -epoch 10 -lr 0.1 -wordNgrams 2

In [74]:
!fasttext test category_model.bin fasttext_dataset_test.txt

- 0.305

In [75]:
!fasttext supervised -input fasttext_dataset_training.txt -output category_model -epoch 10 -lr 2.0 -wordNgrams 2

In [76]:
!fasttext test category_model.bin fasttext_dataset_test.txt

# Grid Search

In [ ]:
!./gridSearch.sh

# Grid Search get results into dataframe (TODO/ WIP)

In [2]:
#get all model's folders
addr ="results/"
modelsFolders = list(os.listdir(addr))
modelsFolders.sort()
print (modelsFolders)
modelsFolders = [ str(mf) for mf in modelsFolders ]

def parseScores(fileaddr):
    scores = {}
    with open(fileaddr, 'r') as f:
        content = list(f.readlines())
        for line in content:
            temp = line.split('\t')
            scores[temp[0].strip()] = float(temp[1].strip())

#change it so key is mf, value is another dict without intermediate keys
def removeSecondLevelIndirectionFromDict(d):
    ret = {}
    for key1 in d: #keep this one
        ret[key1] = {}
        for key2 in d[key1]:
            for key3 in d[key1][key2]:
                ret[key1][key3] = d[key1][key2][key3]

    scores = removeSecondLevelIndirectionFromDict(scores)
    return scores

def parsePars(fileaddr):
    pars = {}
    with open(fileaddr, 'r') as f:
        content = list(f.readlines())
        for line in content:
            temp = line[1:-2]
            temp = temp.split('\\n')
            temp = [ t.strip() for t in temp if t.strip() != '' ]
            for t in temp:
                split = t.split(':')
                pars[split[0].strip()] = split[1].strip()
                try:
                    pars[split[0].strip()] = float(pars[split[0].strip()])
                except:
                    continue
    return pars
    
scores = {} #key: mf (string), value, another dict of scores and their values
pars = {}
for mf in modelsFolders:
    print (mf)
    modelFiles = os.listdir(addr+mf)
    scoreFiles = [ mf for mf in modelFiles if 'At1' in mf and 'test' in mf ] #isolate the test files produced by fasttext which contain the scores, using precisions @1 for now
    scoreFiles.sort()
#     print (scoreFiles)
    parFile = 'parList.txt'
    
    #read scores
    scores[mf] = {}
    for scoreFile in scoreFiles:
        scores[mf][scoreFile] = parseScores(addr+mf+'/'+scoreFile)
         
    #read parameters
    pars[mf] = parsePars(addr+mf+'/'+parFile)
    
parsDf = pd.DataFrame.from_dict(pars, orient='index')
parsDf['mf'] = parsDf.index
parsDf['mf'] = parsDf['mf'].astype(int)
print (parsDf.head())

scoresDf = pd.DataFrame.from_dict(scores, orient='index')
scoresDf['mf'] = scoresDf.index
scoresDf['mf'] = scoresDf['mf'].astype(int)
print (scoresDf.head())

scoresDatabase = pd.merge(parsDf, scoresDf, on="mf")
scoresDatabase.to_csv(addr+'scoresDatabase.csv')


['1', '10', '11', '12', '13', '14', '15', '16', '2', '3', '4', '5', '6', '7', '8', '9']
1


IOError: [Errno 2] No such file or directory: 'results/At8.test/At1.test'